# Monitorización del modeloModel Monitor

Este cuaderno vamos a aprender lo siguiente:

* Alojar un modelo en Amazon SageMaker y capturar solicitudes de inferencia, resultados y metadatos.
* Analizar un Dataset de entrenamiento para generar restricciones de referencia
* Supervisar un endpoint desplegado para detectar violaciones de las restricciones


---
## Setup

Para empezar, hemos de que ha completado estos requisitos previos.

* Especificar una región de AWS para alojar el modelo.
* Disponer del ARN de un rol IAM que de acceso a Amazon SageMaker a los datos en Amazon Simple Storage Service (Amazon S3).
* Tener creado un bucket de S3 para almacenar los datos utilizados para entrenar su modelo, cualquier dato adicional del modelo y los datos capturados de las invocaciones del modelo. 


In [ ]:
import boto3
import os
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name
role = get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
prefix = 'tf-2-workflow'

s3_capture_upload_path = f's3://{bucket}/{prefix}/monitoring/datacapture'

reports_prefix = f'{prefix}/reports'
s3_report_path = f's3://{bucket}/{reports_prefix}'

print(f"Capture path: {s3_capture_upload_path}")
print(f"Report path: {s3_report_path}")

# PARTE A: Capturando datos de inferencia en tiempo real Capturing real-time inference data from Amazon SageMaker endpoints

Vamos a crear un endpoint para mostrar la capacidad de captura de datos en acción.


### Despliegue del modelo en Amazon SageMaker

Vamos a comenzar desplegando el modelo entrenado en `notebooks/10-sagemaker-using-sagemaker-apis.ipynb`.

In [ ]:
import boto3

def get_latest_training_job_name(base_job_name):
    client = boto3.client('sagemaker')
    response = client.list_training_jobs(
        NameContains=base_job_name, SortBy='CreationTime', 
        SortOrder='Descending', StatusEquals='Completed'
    )
    
    if len(response['TrainingJobSummaries']) > 0 :
        return response['TrainingJobSummaries'][0]['TrainingJobName']
    
    else:
        raise Exception('Training job not found.')

def get_training_job_s3_model_artifacts(job_name):
    client = boto3.client('sagemaker')
    response = client.describe_training_job(TrainingJobName=job_name)
    s3_model_artifacts = response['ModelArtifacts']['S3ModelArtifacts']
    return s3_model_artifacts

latest_training_job_name = get_latest_training_job_name('tf-2-workflow')
print(latest_training_job_name)
model_path = get_training_job_s3_model_artifacts(latest_training_job_name)
print(model_path)

Aquí creamos el objeto de tipo modelo con la imagen y el modelo

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel

tensorflow_model = TensorFlowModel(
    model_data = model_path,
    role = role,
    framework_version = '2.3.1'
)


Activamos la captura de datos al desplegar el endpoint

In [ ]:
from time import gmtime, strftime
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'tf-2-workflow-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

predictor = tensorflow_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name,
    # Activamos la captura de datos
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=s3_capture_upload_path
    )
)

### Preparación del Dataset

A continuación, importamos el Dataset. El Dataset en sí es pequeño y está relativamente libre de problemas. Por ejemplo, no hay valores nulos, un problema bastante común. Por tanto, el preprocesamiento sólo consta de la normalización de los datos.

In [ ]:
import numpy as np
from tensorflow.python.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

## Invocación del modelo desplegado

Ahora podemos enviar datos a este endpoint para obtener inferencias en tiempo real. Dado que hemos habilitado la captura de datos en los pasos anteriores, el payload de la solicitud y la respuesta, junto con algunos metadatos adicionales, se guardan en la ubicación de S3 que hemos especificado en `DataCaptureConfig`.

This step invokes the endpoint with included sample data for about 3 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

Este paso invoca al endpoint con datos de muestra durante unos 3 minutos. Los datos se capturan en función del porcentaje de muestreo especificado y la captura continúa hasta que se desactiva la opción de captura de datos.

In [ ]:
%%time 

import time

print(f"Sending test traffic to the endpoint {endpoint_name}. \nPlease wait...")

flat_list =[]
for item in x_test:
    result = predictor.predict(item)['predictions'] 
    flat_list.append(float('%.1f'%(np.array(result))))
    time.sleep(1.8)
    
print("Done!")
print(f'predictions: \t{np.array(flat_list)}')

## Visualización de los datos capturados

Ahora se enumeran los archivos de captura de datos almacenados en Amazon S3. Deberíamos ver diferentes archivos de diferentes períodos de tiempo organizados en base a la hora en la que se produjo la invocación. El formato de la ruta de Amazon S3 es `s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`.

<b>La llegada de los datos capturados a Amazon S3 puede tardar un par de minutos, por lo que la siguiente celda podría dar error. Si esto pasa, reintentaremos después de un minuto.</b>

In [ ]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix='tf-2-workflow/monitoring/datacapture/')
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

A continuación, visualizamos el contenido de un solo archivo de captura. Podemos ver todos los datos capturados en un archivo con formato JSON específico de Amazon SageMaker. Echemos un vistazo a las primeras líneas del archivo capturado.

In [ ]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

Por último veamos el contenido de una sola línea en un archivo JSON formateado para que se apreciar observarlo un poco mejor.

In [ ]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

Como puedmos ver, cada solicitud de inferencia se captura en una línea en el archivo JSON. La línea contiene tanto la entrada como la salida combinadas. En el ejemplo proporcionamos el ContentType como `text/csv` que se refleja en el valor `observedContentType`. Además, expusimos el tipo de codificación utilizado para codificar los payloads de entrada y salida en el formato de captura con el valor `encoding`.

Para recapitular, hemos observado cómo habilitar la captura de payloads de entrada o salida a un endpoint con un nuevo parámetro. También hemos observado el aspecto del formato de captura en Amazon S3.

A continuación, vamos a tratar de monitorizar los datos recopilados en Amazon S3.

# PARTE B: Monitorización del modelo. Estableciendo lineas base y monitorización continua

Además de recopilar los datos, Amazon SageMaker proporciona la capacidad de supervisar y evaluar los datos observados por los endpoints. Para ello:

1. Creamos una línea de base con la que comparar el tráfico en tiempo real.
1. Una vez que la línea de base está lista, configuramos un schedule para evaluar y comparar continuamente con la línea de base.

## 1. Sugerencia de restricción con baseline/training Dataset

El Dataset de entrenamiento con el que se ha entrenado el modelo suele ser una buena referencia. Teniendo en mente que el esquema de los datos del Dataset de entrenamiento y el de inferencia deben coincidir exactamente (es decir, el número y el orden de las features).

A partir del Dataset de entrenamiento podemos pedir a SageMaker que sugiera un conjunto de "restricciones" de referencia y genere "estadísticas" descriptivas para explorar los datos. Para este ejemplo, subiremos el Dataset de entrenamiento que se utilizó para entrenar el modelo preentrenado. Si ya lo tienes en Amazon S3, puedes apuntar directamente a él.

### Preparación del Dataset de entrenamiento con cabeceras

In [ ]:
import pandas as pd
dt = pd.DataFrame(
  data = x_train, 
  columns = [
    "CRIM",
    "ZN",
    "INDUS",
    "CHAS",
    "NOX",
    "RM",
    "AGE",
    "DIS",
    "RAD",
    "TAX",
    "PTRATIO",
    "B",
    "LSTAT"
  ]
)

dt.to_csv("training-dataset-with-header.csv", index = False)

Copiamos el conjunto de datos de entrenamiento en Amazon S3 (si ya lo tienes en Amazon S3, puedes reutilizarlo)

In [ ]:
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = f's3://{bucket}/{baseline_data_prefix}'
baseline_results_uri = f's3://{bucket}/{baseline_results_prefix}'
print(f'Baseline data uri: {baseline_data_uri}')
print(f'Baseline results uri: {baseline_results_uri}')


In [ ]:
training_data_file = open("training-dataset-with-header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'training-dataset-with-header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

### Crear un job para establecer una línea de referencia con el Dataset de entrenamiento

Now that you have the training data ready in Amazon S3, start a job to `suggest` constraints. `DefaultModelMonitor.suggest_baseline(..)` starts a `ProcessingJob` using an Amazon SageMaker provided Model Monitor container to generate the constraints.

Ahora que tenemos los datos de entrenamiento listos en S3, iniciamos un job para `sugerir` restricciones. `DefaultModelMonitor.suggest_baseline(..)` inicia un `ProcessingJob` utilizando un contenedor de Model Monitor proporcionado por SageMaker para generar las restricciones.

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/training-dataset-with-header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

### Exploración de las restricciones generadas y las estadísticas

In [ ]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

In [ ]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

In [ ]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

## 2. Analizando los datos recolectados para problemas de calidad en los datos

### Creación de un schedule

Podemos crear un schedule de supervisión del modelo para el endpoint creado anteriormente. Utilizaremos los recursos de referencia (restricciones y estadísticas) para comparar con el tráfico en tiempo real.

En el análisis anterior, hemos visto cómo se guardan los datos capturados - ese es el formato estándar de entrada y salida para los modelos Tensorflow. Pero Model Monitor es agnóstico al framework, y espera un formato [explicado en la documentación](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-and-post-processing.html#model-monitor-pre-processing-script):

```json
- Input
    - Flattened JSON `{"feature0": <value>, "feature1": <value>...}`
    - Tabular `"<value>, <value>..."`
- Output:
    - Flattened JSON `{"prediction0": <value>, "prediction1": <value>...}`
    - Tabular `"<value>, <value>..."`
```

Necesitamos transformar los registros de entrada para cumplir con este requisito. Model Monitor ofrece _scripts de preprocesamiento_ en Python para transformar la entrada. La celda de abajo tiene el script que funcionará para nuestro caso.

In [ ]:
%%writefile preprocessing.py

import json

def preprocess_handler(inference_record):
    input_data = json.loads(inference_record.endpoint_input.data)
    input_data = {f"feature{i}": val for i, val in enumerate(input_data)}
    
    output_data = json.loads(inference_record.endpoint_output.data)["predictions"][0][0]
    output_data = {"prediction0": output_data}
    
    return{**input_data}

Subiremos este script a un s3 y lo pasaremos como el parámetro `record_preprocessor_script` de la llamada `create_monitoring_schedule`.

In [ ]:
preprocessor_s3_dest_path = f"s3://{bucket}/{prefix}/artifacts/modelmonitor"
preprocessor_s3_dest = sagemaker.s3.S3Uploader.upload("preprocessing.py", preprocessor_s3_dest_path)
print(preprocessor_s3_dest)

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'DEMO-tf-2-workflow-model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    record_preprocessor_script=preprocessor_s3_dest,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

### Generando violaciones de las restricciones artificialmente

Para obtener algún resultado relevante para el análisis de la monitorización, podemos  generar artificialmente algunas inferencias con valores de features que causen violaciones específicas, y luego invocar el endpoint con estos datos.

Mirando nuestras características RM y AGE:

- RM - número medio de habitaciones por vivienda
- AGE - proporción de viviendas ocupadas por sus propietarios construidas antes de 1940

Vamos a simular una situación en la que el número medio de habitaciones y la proporción de viviendas ocupadas por sus propietarios son ambos -10.

In [ ]:
df_with_violations = pd.read_csv("training-dataset-with-header.csv")
df_with_violations["RM"] = -10
df_with_violations["AGE"] = -10
df_with_violations

### Generar tráfico artificial

La celda de abajo inicia un hilo para enviar tráfico al endpoint. Hay que tener en cuenta  que necesitaremos detener el kernel para terminar este hilo. Si no hay tráfico, los trabajos de monitorización se marcan como `Failed` ya que no hay datos que procesar.

In [ ]:
from threading import Thread
from time import sleep
import time

def invoke_endpoint():
    for item in df_with_violations.to_numpy():
        result = predictor.predict(item)['predictions'] 
        time.sleep(0.5)

def invoke_endpoint_forever():
    while True:
        invoke_endpoint()
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Hay que parar el kernel para detener las invocaciones al endpoint

### Describir e inspeccionar el schedule

Una vez que describimos, observemos que `MonitoringScheduleStatus` cambia a `Scheduled`.

In [ ]:
desc_schedule_result = my_default_monitor.describe_schedule()
print(f"Schedule status: {desc_schedule_result['MonitoringScheduleStatus']}")

### Listar ejecuciones

El schedule inicia los jobs en los intervalos previamente especificados. Vamos a enumerar las últimas cinco ejecuciones. Hay que tener en cuenta que si lanzamos esto después de crear el schedule, las ejecuciones podrían ser nulas. Es posible que tengamos que esperar a cruzar el límite de la hora (en UTC) para ver las ejecuciones iniciadas. El código de abajo implementa la lógica para esperar.

Nota: Incluso para una programación horaria, SageMaker tiene un periodo de buffer de 20 minutos para programar su ejecución. Podemos comprobar ver que nuestra ejecución comienza en cualquier momento entre cero y ~20 minutos desde el límite de la hora. Esto es controlado y se hace para equilibrar la carga en el backend.

In [ ]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()    

### Inspección de una ejecución específica (última)


En la celda anterior, se recogió la última ejecución programada completada o fallida. Aquí están los posibles estados de finalización y lo que significa cada uno de ellos:

* Completed - Esto significa que la ejecución de monitoreo se completó y no se encontraron problemas en el informe de violaciones.
* CompletedWithViolations - Esto significa que la ejecución se completó, pero se detectaron violaciones de las restricciones.
* Failed - La ejecución de monitoreo falló, tal vez debido a un error del cliente (tal vez permisos incorrectas del rol ) o problemas de infraestructura. Es necesario un examen más detallado de `FailureReason` y `ExitMessage` para identificar qué ocurrió exactamente.
* Stopped: el trabajo superó el tiempo máximo de ejecución o se detuvo manualmente.

In [ ]:
latest_execution = mon_executions[-1] # el index de la  última ejecución es -1, penúltimo es -2 ...
#time.sleep(60)
latest_execution.wait(logs=False)

print(f"Latest execution status: {latest_execution.describe()['ProcessingJobStatus']}")
print(f"Latest execution result: {latest_execution.describe()['ExitMessage']}")

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

In [ ]:
report_uri=latest_execution.output.destination
print(f'Report Uri: {report_uri}')

### Listando los informes generados

In [ ]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print(f'Report bucket: {report_bucket}')
print(f'Report key: {report_key}')

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

### Informe de violaciones

Si hay alguna violación en comparación con la línea de base, se indicará aquí.

In [ ]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

### Disparando la ejecución manualmente

Para disparar la ejecución manualmente, primero obtenemos todas las rutas de captura de datos, estadísticas de línea base, restricciones de línea base, etc. A continuación, utilizamos la función siguiente para ejecutar el trabajo de procesamiento.

In [ ]:
%%writefile monitoringjob_utils.py

import os, sys
from urllib.parse import urlparse
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

def get_model_monitor_container_uri(region):
    container_uri_format = '{0}.dkr.ecr.{1}.amazonaws.com/sagemaker-model-monitor-analyzer'
    
    regions_to_accounts = {
        'eu-north-1': '895015795356',
        'me-south-1': '607024016150',
        'ap-south-1': '126357580389',
        'us-east-2': '680080141114',
        'us-east-2': '777275614652',
        'eu-west-1': '468650794304',
        'eu-central-1': '048819808253',
        'sa-east-1': '539772159869',
        'ap-east-1': '001633400207',
        'us-east-1': '156813124566',
        'ap-northeast-2': '709848358524',
        'eu-west-2': '749857270468',
        'ap-northeast-1': '574779866223',
        'us-west-2': '159807026194',
        'us-west-1': '890145073186',
        'ap-southeast-1': '245545462676',
        'ap-southeast-2': '563025443158',
        'ca-central-1': '536280801234'
    }
    
    container_uri = container_uri_format.format(regions_to_accounts[region], region)
    return container_uri

def get_file_name(url):
    a = urlparse(url)
    return os.path.basename(a.path)

def run_model_monitor_job_processor(
    region, instance_type, role, data_capture_path, statistics_path,
    constraints_path, reports_path, instance_count=1, preprocessor_path=None,
    postprocessor_path=None, publish_cloudwatch_metrics='Disabled'
):    
    data_capture_sub_path = data_capture_path[data_capture_path.rfind('datacapture/') :]
    data_capture_sub_path = data_capture_sub_path[data_capture_sub_path.find('/') + 1 :]
    processing_output_paths = reports_path + '/' + data_capture_sub_path
    
    input_1 = ProcessingInput(
        input_name='input_1',
        source=data_capture_path,
        destination='/opt/ml/processing/input/endpoint/' + data_capture_sub_path,
        s3_data_type='S3Prefix',
        s3_input_mode='File'
    )

    baseline = ProcessingInput(
        input_name='baseline',
        source=statistics_path,
        destination='/opt/ml/processing/baseline/stats',
        s3_data_type='S3Prefix',
        s3_input_mode='File'
    )

    constraints = ProcessingInput(
        input_name='constraints',
        source=constraints_path,
        destination='/opt/ml/processing/baseline/constraints',
        s3_data_type='S3Prefix',
        s3_input_mode='File'
    )

    outputs = ProcessingOutput(
        output_name='result',
        source='/opt/ml/processing/output',
        destination=processing_output_paths,
        s3_upload_mode='Continuous'
    )

    env = {
        'baseline_constraints': '/opt/ml/processing/baseline/constraints/' + get_file_name(constraints_path),
        'baseline_statistics': '/opt/ml/processing/baseline/stats/' + get_file_name(statistics_path),
        'dataset_format': '{"sagemakerCaptureJson":{"captureIndexNames":["endpointInput","endpointOutput"]}}',
        'dataset_source': '/opt/ml/processing/input/endpoint',
        'output_path': '/opt/ml/processing/output',
        'publish_cloudwatch_metrics': publish_cloudwatch_metrics }
    
    inputs=[input_1, baseline, constraints]
    
    if postprocessor_path:
        env['post_analytics_processor_script'] = '/opt/ml/processing/code/postprocessing/' + get_file_name(postprocessor_path)
        
        post_processor_script = ProcessingInput(
            input_name='post_processor_script',
            source=postprocessor_path,
            destination='/opt/ml/processing/code/postprocessing',
            s3_data_type='S3Prefix',
            s3_input_mode='File'
        )
        inputs.append(post_processor_script)

    if preprocessor_path:
        env['record_preprocessor_script'] = '/opt/ml/processing/code/preprocessing/' + get_file_name(preprocessor_path)

        pre_processor_script = ProcessingInput(
            input_name='pre_processor_script',
            source=preprocessor_path,
            destination='/opt/ml/processing/code/preprocessing',
            s3_data_type='S3Prefix',
            s3_input_mode='File'
        )
        inputs.append(pre_processor_script) 
    
    processor = Processor(
        image_uri = get_model_monitor_container_uri(region),
        instance_count = instance_count,
        instance_type = instance_type,
        role=role,
        env = env
    )

    return processor.run(inputs=inputs, outputs=[outputs])

In [ ]:
result = s3_client.list_objects(Bucket=bucket, Prefix='tf-2-workflow/monitoring/datacapture/')
capture_files = [f's3://{bucket}/{capture_file.get("Key")}' for capture_file in result.get('Contents')]

print("Capture Files: ")
print("\n".join(capture_files))

data_capture_path = capture_files[len(capture_files) - 1][: capture_files[len(capture_files) - 1].rfind('/')]
statistics_path = baseline_results_uri + '/statistics.json'
constraints_path = baseline_results_uri + '/constraints.json'

print(data_capture_path)
print(preprocessor_s3_dest)
print(statistics_path)
print(constraints_path)
print(s3_report_path)

In [ ]:
from monitoringjob_utils import run_model_monitor_job_processor

processor = run_model_monitor_job_processor(
    region, 'ml.m5.xlarge', 
    role, 
    data_capture_path, 
    statistics_path, 
    constraints_path, 
    s3_report_path,
    preprocessor_path=preprocessor_s3_dest
)

### Inspección de la ejecución

In [ ]:
import boto3

def get_latest_model_monitor_processing_job_name(base_job_name):
    client = boto3.client('sagemaker')
    response = client.list_processing_jobs(
        NameContains=base_job_name,
        SortBy='CreationTime', 
        SortOrder='Descending',
        StatusEquals='Completed'
    )
    
    if len(response['ProcessingJobSummaries']) > 0 :
        return response['ProcessingJobSummaries'][0]['ProcessingJobName']
    else:
        raise Exception('Processing job not found.')

def get_model_monitor_processing_job_s3_report(job_name):
    client = boto3.client('sagemaker')
    response = client.describe_processing_job(ProcessingJobName=job_name)
    s3_report_path = response['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
    return s3_report_path

latest_model_monitor_processing_job_name = get_latest_model_monitor_processing_job_name('sagemaker-model-monitor-analyzer')
print(latest_model_monitor_processing_job_name)
report_path = get_model_monitor_processing_job_s3_report(latest_model_monitor_processing_job_name)
print(report_path)

In [ ]:
pd.set_option('display.max_colwidth', -1)
df = pd.read_json('{}/constraint_violations.json'.format(report_path))
df

## Borrar los recursos

Puedes mantener el endpoint en funcionamiento para seguir capturando datos. Si no tienes previsto recopilar más datos ni seguir utilizando este endpoint, debe eliminarlo para evitar incurrir en cargos adicionales. Ten en cuenta que la eliminación del endpoint no elimina los datos que se capturaron durante las invocaciones del modelo. Esos datos persisten en Amazon S3 hasta los elimines.

Pero antes de eso, debes eliminar primero el schedule.

In [ ]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(120) # Esperar hasta que se borre definitivamente

In [ ]:
predictor.delete_endpoint()